In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [4]:
def load_dataset(file_path):
    return pd.read_sas(file_path, format="xport")


In [26]:
bp_data= load_dataset("BPXO_L.xpt")
bp_data

,SEQN,BPAOARM,BPAOCSZ,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3
0,130378.0,b'R',4.0,135.0,98.0,131.0,96.0,132.0,94.0,82.0,79.0,82.0
1,130379.0,b'R',4.0,121.0,84.0,117.0,76.0,113.0,76.0,72.0,71.0,73.0
2,130380.0,b'R',4.0,111.0,79.0,112.0,80.0,104.0,76.0,84.0,83.0,77.0
3,130386.0,b'R',4.0,110.0,72.0,120.0,74.0,115.0,75.0,59.0,64.0,64.0
4,130387.0,b'R',4.0,143.0,76.0,136.0,74.0,145.0,78.0,80.0,80.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7796,142306.0,b'R',2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7797,142307.0,b'R',4.0,127.0,75.0,132.0,73.0,131.0,72.0,71.0,70.0,67.0
7798,142308.0,b'R',3.0,106.0,65.0,106.0,69.0,112.0,74.0,58.0,61.0,69.0
7799,142309.0,b'R',3.0,127.0,81.0,125.0,82.0,128.0,81.0,80.0,79.0,83.0


In [35]:
demo_data= load_dataset("DEMO_L.xpt")
demo_data

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,INDFMPIR
0,130378.0,12.0,2.0,1.0,43.0,NaN,5.0,6.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,50055.450807,54374.463898,173.0,2.0,5.00
1,130379.0,12.0,2.0,1.0,66.0,NaN,3.0,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,29087.450605,34084.721548,173.0,2.0,5.00
2,130380.0,12.0,2.0,2.0,44.0,NaN,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,80062.674301,81196.277992,174.0,1.0,1.41
3,130381.0,12.0,2.0,2.0,5.0,NaN,5.0,7.0,1.0,71.0,...,2.0,2.0,2.0,3.0,NaN,38807.268902,55698.607106,182.0,2.0,1.53
4,130382.0,12.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,34.0,...,2.0,2.0,3.0,1.0,2.0,30607.519774,36434.146346,182.0,2.0,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11928,142306.0,12.0,2.0,1.0,9.0,NaN,2.0,2.0,1.0,111.0,...,1.0,3.0,3.0,3.0,NaN,11147.192563,13459.129019,176.0,1.0,2.01
11929,142307.0,12.0,2.0,2.0,49.0,NaN,4.0,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,69419.620456,64962.328962,181.0,1.0,NaN
11930,142308.0,12.0,2.0,1.0,50.0,NaN,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,32696.313477,44367.534132,183.0,2.0,1.95
11931,142309.0,12.0,2.0,1.0,40.0,NaN,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,30547.974564,46249.361849,176.0,1.0,3.11


In [39]:
fertin_data= load_dataset("FERTIN_L.xpt")
fertin_data

,SEQN,WTPH2YR,LBXFER,LBDFERSI
0,130380.0,8.532884e+04,13.30,13.30
1,130381.0,5.397605e-79,NaN,NaN
2,130382.0,5.963893e+04,24.40,24.40
3,130388.0,3.286429e+04,NaN,NaN
4,130390.0,6.142701e+04,52.10,52.10
...,...,...,...,...
2559,142280.0,6.639765e+04,66.40,66.40
2560,142282.0,5.397605e-79,NaN,NaN
2561,142283.0,1.527882e+05,31.60,31.60
2562,142300.0,3.147068e+04,5.69,5.69


In [12]:
insurance_data= load_dataset("HIQ_L.xpt")
insurance_data = insurance_data.drop(['HIQ032C', 'HIQ032E', 'HIQ032F','HIQ032H','HIQ032I' ], axis=1)
insurance_data = insurance_data.dropna()
insurance_data

,SEQN,HIQ011,HIQ032A,HIQ032B,HIQ032D,HIQ210
53,130431.0,1.0,1.0,2.0,4.0,2.0
426,130804.0,1.0,1.0,2.0,4.0,2.0
799,131177.0,1.0,1.0,2.0,4.0,2.0
1417,131795.0,1.0,1.0,2.0,4.0,2.0
1520,131898.0,1.0,1.0,2.0,4.0,2.0
2094,132472.0,1.0,1.0,2.0,4.0,2.0
2686,133064.0,1.0,1.0,2.0,4.0,2.0
2893,133271.0,1.0,1.0,2.0,4.0,2.0
2996,133374.0,1.0,1.0,2.0,4.0,2.0
3348,133726.0,1.0,1.0,2.0,4.0,2.0


In [15]:
creactive_data= load_dataset("HSCRP_L.xpt")
creactive_data= creactive_data.dropna()
creactive_data


,SEQN,WTPH2YR,LBXHSCRP,LBDHRPLC
0,130378.0,56042.129410,1.78,5.397605e-79
1,130379.0,37435.705647,2.03,5.397605e-79
2,130380.0,85328.844519,5.62,5.397605e-79
5,130386.0,44526.214135,1.05,5.397605e-79
6,130387.0,22746.296353,3.96,5.397605e-79
...,...,...,...,...
8719,142303.0,47778.604939,2.15,5.397605e-79
8721,142305.0,49710.929024,2.50,5.397605e-79
8723,142307.0,68994.175834,4.30,5.397605e-79
8725,142309.0,46284.416719,0.53,5.397605e-79


In [10]:
kidneyCondition_data = load_dataset("KIQ_U_L.xpt")
kidneyCondition_data = kidneyCondition_data.drop(['KIQ025', 'KIQ010', 'KIQ052'], axis=1)
kidneyCondition_data = kidneyCondition_data.dropna()
kidneyCondition_data

,SEQN,KIQ022,KIQ005,KIQ042,KIQ044,KIQ481
1,130379.0,2.0,2.0,1.0,2.0,2.000000e+00
2,130380.0,2.0,1.0,2.0,2.0,2.000000e+00
5,130386.0,2.0,1.0,2.0,2.0,1.000000e+00
6,130387.0,2.0,1.0,2.0,2.0,3.000000e+00
8,130389.0,2.0,1.0,2.0,1.0,1.000000e+00
...,...,...,...,...,...,...
7803,142303.0,2.0,5.0,1.0,1.0,3.000000e+00
7805,142307.0,2.0,5.0,1.0,1.0,5.397605e-79
7806,142308.0,2.0,1.0,2.0,2.0,5.397605e-79
7807,142309.0,2.0,1.0,2.0,2.0,2.000000e+00


In [8]:
kidneyCondition_data.describe()

,SEQN,KIQ022,KIQ005,KIQ042,KIQ044,KIQ481
count,7809.000000,7807.000000,5208.000000,5201.000000,5193.000000,5.187000e+03
mean,136369.526316,1.970539,2.007104,1.670063,1.646255,1.528244e+00
std,3441.457300,0.349082,1.350639,0.503417,0.516516,1.307415e+00
min,130378.000000,1.000000,1.000000,1.000000,1.000000,5.397605e-79
25%,133382.000000,2.000000,1.000000,1.000000,1.000000,1.000000e+00
50%,136410.000000,2.000000,1.000000,2.000000,2.000000,1.000000e+00
75%,139337.000000,2.000000,3.000000,2.000000,2.000000,2.000000e+00
max,142310.000000,9.000000,9.000000,9.000000,9.000000,9.000000e+00
